In [1]:
import requests
import warcio
from contextlib import closing
from bs4 import BeautifulSoup
import json

import logging
from IPython.display import HTML
import pandas as pd

In [2]:
websites = pd.read_csv("./ccmain/ccmain_2021-04_long.txt", header = None, sep = '|') 

In [3]:
websites.head()

,0,1,2,3,4
0,NaN,globalannal.wordpress.com ...,/tag/careers/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
1,NaN,globalannal.wordpress.com ...,/tag/jobs-numbers/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
2,NaN,globalannal.wordpress.com ...,/tag/jobs-report/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...,NaN
3,NaN,globalshapersbucharest.wordpress.com ...,/2018/04/11/programs-officer-job-global-shaper...,crawl-data/CC-MAIN-2021-04/segments/1610703506...,NaN
4,NaN,globalyouthbook.wordpress.com ...,/tag/career-planning/ ...,crawl-data/CC-MAIN-2021-04/segments/1610704798...,NaN


In [4]:
websites.columns

Int64Index([0, 1, 2, 3, 4], dtype='int64')

In [5]:
websites.drop([0, 4], axis=1, inplace=True)
websites.head()

,1,2,3
0,globalannal.wordpress.com ...,/tag/careers/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,globalannal.wordpress.com ...,/tag/jobs-numbers/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,globalannal.wordpress.com ...,/tag/jobs-report/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,globalshapersbucharest.wordpress.com ...,/2018/04/11/programs-officer-job-global-shaper...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,globalyouthbook.wordpress.com ...,/tag/career-planning/ ...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [6]:
websites.columns = ['url_host','url_path', 'warc_file']
websites.head()

,url_host,url_path,warc_file
0,globalannal.wordpress.com ...,/tag/careers/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
1,globalannal.wordpress.com ...,/tag/jobs-numbers/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
2,globalannal.wordpress.com ...,/tag/jobs-report/ ...,crawl-data/CC-MAIN-2021-04/segments/1610703528...
3,globalshapersbucharest.wordpress.com ...,/2018/04/11/programs-officer-job-global-shaper...,crawl-data/CC-MAIN-2021-04/segments/1610703506...
4,globalyouthbook.wordpress.com ...,/tag/career-planning/ ...,crawl-data/CC-MAIN-2021-04/segments/1610704798...


In [7]:
websites.url_host.value_counts()

www.workana.com                                              14148
www.workopolis.com                                            6140
www.worknrby.com                                              2827
www.workinsports.com                                          2458
seeker.worksourcewa.com                                       2078
                                                             ...  
rheingoldblog.wordpress.com                                      1
remissartist.wordpress.com                                       1
tomasclancy.wordpress.com                                        1
sonibindaas571181659.wordpress.com                               1
readneckreview.wordpress.com                                     1
Name: url_host, Length: 7345, dtype: int64

In [8]:
websites.url_host.value_counts().keys()

Index(['www.workana.com                                          ',
       'www.workopolis.com                                       ',
       'www.worknrby.com                                         ',
       'www.workinsports.com                                     ',
       'seeker.worksourcewa.com                                  ',
       'kwoted.wordpress.com                                     ',
       'www.xn--jobbrse-stellenangebote-blc.de                   ',
       'www.workitdaily.com                                      ',
       'workinhostels.com                                        ',
       'workshift-sol.com                                        ',
       ...
       'paulineobrien.wordpress.com                              ',
       'kalailmforlove.wordpress.com                             ',
       'lowmanhenry.wordpress.com                                ',
       'bkk-vdn.vion-online.de                                   ',
       'talkiesnetwork.wordpress.com 

In [9]:
hostList = []
for i in websites.url_host.value_counts().keys():
    hostList.append(i.replace(' ',''))
    
print(hostList)

['www.workana.com', 'www.workopolis.com', 'www.worknrby.com', 'www.workinsports.com', 'seeker.worksourcewa.com', 'kwoted.wordpress.com', 'www.xn--jobbrse-stellenangebote-blc.de', 'www.workitdaily.com', 'workinhostels.com', 'workshift-sol.com', 'www.workinaustria.com', 'www.xn--jobs-fr-bottrop-4vb.de', 'www.venquis.de', 'www.workwithglee.com', 'www.xn--jobs-fr-bochum-lsb.de', 'www.world-working.com', 'workfa.com', 'www.work4a.com', 'workinstartups.com', 'resources.workable.com', 'jobs.workinoptics.com', 'www.workventure.com', 'www.xn--jobs-fr-aschaffenburg-eic.de', 'wirsuchenmenschen.de', 'www.workpac.com', 'www.worky.com', 'wirsuchenwiedereinsteiger.de', 'www.wunschkandidaten.de', 'www.workcaregiver.com', 'www.workpcb.com', 'johnnyvoid.wordpress.com', 'www.worksourcewa.com', 'www.xn--jobs-fr-bayreuth-ozb.de', 'worknowapp.com', 'wir-suchen-menschen.de', 'www.xn--jobs-fr-bamberg-4vb.de', 'www.xn--jobs-fr-braunschweig-uec.de', 'wirsuchenheilerziehungspfleger.de', 'literatureismyporn.wordp

In [10]:
!python -m pip install cdx_toolkit

In [11]:
import cdx_toolkit

In [12]:
cdx = cdx_toolkit.CDXFetcher(source='cc')

In [13]:
cdx_indexes = requests.get('https://index.commoncrawl.org/collinfo.json').json()

In [14]:
pd.options.display.max_colwidth=150
pd.options.display.max_rows=6

In [15]:
pd.DataFrame(cdx_indexes)

,id,name,timegate,cdx-api
0,CC-MAIN-2021-04,January 2021 Index,https://index.commoncrawl.org/CC-MAIN-2021-04/,https://index.commoncrawl.org/CC-MAIN-2021-04-index
1,CC-MAIN-2020-50,November 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-50/,https://index.commoncrawl.org/CC-MAIN-2020-50-index
2,CC-MAIN-2020-45,October 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-45/,https://index.commoncrawl.org/CC-MAIN-2020-45-index
...,...,...,...,...
75,CC-MAIN-2012,Index of 2012 ARC files,https://index.commoncrawl.org/CC-MAIN-2012/,https://index.commoncrawl.org/CC-MAIN-2012-index
76,CC-MAIN-2009-2010,Index of 2009 - 2010 ARC files,https://index.commoncrawl.org/CC-MAIN-2009-2010/,https://index.commoncrawl.org/CC-MAIN-2009-2010-index
77,CC-MAIN-2008-2009,Index of 2008 - 2009 ARC files,https://index.commoncrawl.org/CC-MAIN-2008-2009/,https://index.commoncrawl.org/CC-MAIN-2008-2009-index


In [16]:
print(pd.DataFrame(cdx_indexes).head(20).to_markdown())

|    | id              | name                 | timegate                                       | cdx-api                                             |
|---:|:----------------|:---------------------|:-----------------------------------------------|:----------------------------------------------------|
|  0 | CC-MAIN-2021-04 | January 2021 Index   | https://index.commoncrawl.org/CC-MAIN-2021-04/ | https://index.commoncrawl.org/CC-MAIN-2021-04-index |
|  1 | CC-MAIN-2020-50 | November 2020 Index  | https://index.commoncrawl.org/CC-MAIN-2020-50/ | https://index.commoncrawl.org/CC-MAIN-2020-50-index |
|  2 | CC-MAIN-2020-45 | October 2020 Index   | https://index.commoncrawl.org/CC-MAIN-2020-45/ | https://index.commoncrawl.org/CC-MAIN-2020-45-index |
|  3 | CC-MAIN-2020-40 | September 2020 Index | https://index.commoncrawl.org/CC-MAIN-2020-40/ | https://index.commoncrawl.org/CC-MAIN-2020-40-index |
|  4 | CC-MAIN-2020-34 | August 2020 Index    | https://index.commoncrawl.org/CC-MAIN-2020-34/

In [17]:
print(pd.DataFrame(cdx_indexes).tail(20).to_markdown())

|    | id                | name                           | timegate                                         | cdx-api                                               |
|---:|:------------------|:-------------------------------|:-------------------------------------------------|:------------------------------------------------------|
| 58 | CC-MAIN-2015-32   | July 2015 Index                | https://index.commoncrawl.org/CC-MAIN-2015-32/   | https://index.commoncrawl.org/CC-MAIN-2015-32-index   |
| 59 | CC-MAIN-2015-27   | June 2015 Index                | https://index.commoncrawl.org/CC-MAIN-2015-27/   | https://index.commoncrawl.org/CC-MAIN-2015-27-index   |
| 60 | CC-MAIN-2015-22   | May 2015 Index                 | https://index.commoncrawl.org/CC-MAIN-2015-22/   | https://index.commoncrawl.org/CC-MAIN-2015-22-index   |
| 61 | CC-MAIN-2015-18   | April 2015 Index               | https://index.commoncrawl.org/CC-MAIN-2015-18/   | https://index.commoncrawl.org/CC-MAIN-2015-18-index   

In [18]:
api_url = cdx_indexes[0]['cdx-api']
api_url

'https://index.commoncrawl.org/CC-MAIN-2021-04-index'

In [19]:
f = open(r"ccmain_2021jan_jobURL.txt","a+")
for url_host in hostList:
    r = requests.get(api_url,
                 params = {
                     'url': url_host,
                     'output': 'json',
                     'filter': ['=status:200', '=mime-detected:text/html']
                 })
    records = [json.loads(line) for line in r.text.split('\n') if line]
    df = pd.DataFrame(records)
    print(url_host,' ',df.shape[0])
    L = [str(url_host),' ',str(df.shape[0]), '\n']
    f.writelines(L) 
f.close()

www.workana.com   55
www.workopolis.com   1
www.worknrby.com   1
www.workinsports.com   1
seeker.worksourcewa.com   1
kwoted.wordpress.com   1
www.xn--jobbrse-stellenangebote-blc.de   5
www.workitdaily.com   1
workinhostels.com   1
workshift-sol.com   2
www.workinaustria.com   1
www.xn--jobs-fr-bottrop-4vb.de   1
www.venquis.de   1
www.workwithglee.com   1
www.xn--jobs-fr-bochum-lsb.de   1
www.world-working.com   2
workfa.com   1
www.work4a.com   2
workinstartups.com   2
resources.workable.com   1
jobs.workinoptics.com   1
www.workventure.com   1
www.xn--jobs-fr-aschaffenburg-eic.de   1
wirsuchenmenschen.de   1
www.workpac.com   1
www.worky.com   1
wirsuchenwiedereinsteiger.de   1
www.wunschkandidaten.de   1
www.workcaregiver.com   1
www.workpcb.com   1
johnnyvoid.wordpress.com   1
www.worksourcewa.com   2
www.xn--jobs-fr-bayreuth-ozb.de   1
worknowapp.com   1
wir-suchen-menschen.de   1
www.xn--jobs-fr-bamberg-4vb.de   1
www.xn--jobs-fr-braunschweig-uec.de   1
wirsuchenheilerziehungspf

www.versorgungskammer.de   4
archive.world-challenge.com   1
www.process.vogel.de   1
www.vk-bodyfit.de   1
wiesbadenrzieht.de   1
forum.wordreference.com   3
workstickers.com   1
www.workhappymums.com   1
wittmedia.de   1
scientistseessquirrel.wordpress.com   1
laviemathematique.wordpress.com   1
www.vip-vitalisten.de   1
varjager.wordpress.com   1
www.workhoppers.com   1
worldconnection.com   4
www.elektronikpraxis.vogel.de   3
occupationoforegon.wordpress.com   1
www.xn--jobbrse-d1a.de   1
www.works-i.com   1
worknhour.com   1
www.xn--jobs-fr-bremen-lsb.de   1
www.versicherungskarrieren.de   2
israbib.wordpress.com   1
womanbitesdog.wordpress.com   1
nicoleandmaggie.wordpress.com   1
wirsindderwandel.de   2
worklifefeed.com   6
workmetall.com   1
lafemmeexec383545705.wordpress.com   1
cdn2.world-architects.com   1
wordstodeeds.com   1
www.workindentistry.com   2
ryanmacdonald1.wordpress.com   1
work-question.com   1
www.widas.de   1
wilabonn.de   1
verbraucherschutz.de   1
www.world

voiio.de   2
shermanave.wordpress.com   1
workathome523.com   1
workhap.com   2
www.work-in-hessen.com   1
www.volksbank-heilbronn.de   1
www.wikway.de   1
hibernationnow.wordpress.com   1
workusainc.com   2
www.workjam.com   6
www.worksolutionstoday.com   1
stayathomeyeti.wordpress.com   1
snowyspace.wordpress.com   1
www.workinsurgery.com   2
www.xn--bag-mdchenhuser-4kbg.de   1
workliferomance.de   2
sveaskog.workbuster.com   1
thehotline.workbrightats.com   1
www.wuv.de   1
workathometipsonline.com   2
mbbsguide.wordpress.com   1
owenzidar.wordpress.com   1
www.wila-arbeitsmarkt.de   1
workalpha.com   1
www.vertriebsjobs.de   1
www.versicherungs-stellen.de   1
homeschoolstrong.wordpress.com   3
visiona.de   2
wirtschaftsfoerderung-sbh.de   1
www.worlddentalnetwork.com   1
worksindubai.com   1
work473.com   1
www.workscene.com   2
www.workservicescorp.com   1
www.velototal.de   1
quotesthoughtsrandom.wordpress.com   1
workforcesolutionsrca.com   2
realizebeauty.wordpress.com   1
www.

workfromhomemoneymiracle.com   1
verbandsjobs.de   1
tcagley.wordpress.com   1
katdesigner.wordpress.com   1
kjellemann.wordpress.com   1
workandfamily.de   3
www.why.de   5
www.workfromhomewatchdog.com   1
mdcpsd2.wordpress.com   1
workremotelyeurope.com   2
www.xn--jobs-fr-bremerhaven-bbc.de   1
www.workforcepartnership.com   1
www.world4gaming.com   1
forms.workday.com   1
www.wir-machen-schramberg.de   1
www.wilabonn.de   1
corporate.windeln.de   1
kurtborne.wordpress.com   1
wvd-dialog-marketing.de   1
milliondollarbuzz.wordpress.com   1
www.xn--jobbrse-mint-7ib.de   1
www.wir-schwarmstedt.de   1
astrowoche.wunderweib.de   1
www.vidia-kliniken.de   1
www.workshopper.com   1
jvaconsulting.wordpress.com   1
vincendo.de   1
workremotelyfromhome.com   1
hunan.workpcb.com   1
www.workforcebulletin.com   1
voicler.de   1
www.worklifedubai.com   1
uwe-repository.worktribe.com   2
www.workthesystem.com   1
afm.vera-via.de   1
www.workspaceinteriors.com   2
travellingmysteryguest.wordpress

jeffsmithmp556572174.wordpress.com   1
workshopswithwow.com   1
jakeasaurasrex.wordpress.com   1
www.vistra-ing.de   2
youngindiachallenge.wordpress.com   1
popmoderne.wordpress.com   1
rebeccaluellamiller.wordpress.com   1
toklandet.wordpress.com   1
workmenandtalents.com   1
world-news-network.com   1
vitamin11.de   1
www.workerscompattorneysatlanta.com   1
thejasbirsingh.wordpress.com   1
motorrad.wmfahrschule.de   1
sportscardinfo.wordpress.com   1
wildfarm-reetz.de   1
wohnen-in-freiberg.de   1
workisto.com   1
mywakenews.wordpress.com   2
www.validas.de   1
wolfganglolies.de   1
neuroinflab.wordpress.com   1
wortmagieblog.wordpress.com   1
www.wuro.de   1
mysatelite.wordpress.com   1
workingperson.com   1
www.winkler.de   1
workplacestaffing.com   1
workingworldcareers.com   1
www.worthmann-ma.de   2
www.vcp.de   4
manicmonday123.wordpress.com   1
www.versicherung-2.de   1
www.xity.de   1
helenmpugsley.wordpress.com   2
www.vodafone-institut.de   3
znrblog.wordpress.com   1
www.v

...

wicke-landschaft.de   1
portraitsofanimals.wordpress.com   1
jarofmothsbooks.wordpress.com   1
www.work-to-not-work.com   1
petersage2.wordpress.com   1
workflowtemp.com   1
www.xn--ambulante-pflege-knzel-9lc.de   1
smsmine.wordpress.com   1
ianmillerblog.wordpress.com   1
wordweavingwriter.com   1
mikkeharry.wordpress.com   1
theplannedjourney.wordpress.com   1
redftech.wordpress.com   1
www.voegeler-bau.de   1
workingmumkitty.com   1
www.witty-karriere.de   1
www.workscapes.com   1
www.viloucosmetics.de   1
thetruthwithnorestriction.wordpress.com   1
www.vianobis-fachklinik.de   2
workaxle.com   1
iranlowooutsourcing.wordpress.com   1
www.whg-ebw.de   1
www.villa-kindertraum.de   1
radeko.wordpress.com   1
verschickungsheime.de   1
oscegar.wordpress.com   1
pennygadd51.wordpress.com   1
shapespl.wordpress.com   1
stevendavisonmd.wordpress.com   1
www.wordswillflow.com   1
themikira.wordpress.com   1
mtdesk.wordpress.com   1
jamesheneghanauthor.wordpress.com   1
members.workrowan.com 

www.wpc-terrassendielen-traumboden24.de   1
raiyawrites.wordpress.com   1
larina.wordpress.com   1
www.whatshealth.de   1
www.wiwe-nw.de   1
pommespa.wordpress.com   1
www.wulf-catering.de   1
www.variatocafe.de   1
stefansphotosweb.wordpress.com   1
ve-k.de   1
www.wolfs-blog.de   1
magasmedia.wordpress.com   1
www.world-of-sun.com   1
susanchalkley.wordpress.com   1
www.workforheadford.com   1
wfg-wnd.de   2
www.workplaceinnovator.com   1
nattlejonet.wordpress.com   1
normsaysno.wordpress.com   1
www.volleyball-verband.de   1
iclassenglish.wordpress.com   1
axel.wfilm.de   1
musanjufukavubu.wordpress.com   1
wkv-online.de   1
oratiocontemplativa.wordpress.com   1
wir-im-landkreis.de   1
rosejasmeen.wordpress.com   1
stefansmek.wordpress.com   1
rajczakowski.wordpress.com   1
westernmeridian.wordpress.com   1
usdynamics365.wordpress.com   1
www.workinglostlovespells.com   1
www.voice-acoustic.de   1
interwatches.wordpress.com   1
stochasticobservations.wordpress.com   1
www.wordyard.c

thestoryfiles.wordpress.com   1
timeseditor.wordpress.com   1
norbertschulze1.wordpress.com   1
thegreenmarketingcompany.wordpress.com   1
jinleisc.world-stone.com   1
taarasite.wordpress.com   1
work-at-home-data-entry.com   1
white-sheep.de   1
vertriebsjobs25.de   1
www.vib-promotion.de   1
www.winzer-sommerach.de   5
nomada2401.wordpress.com   1
www.worldclassind.com   1
munichbeachclub.wordpress.com   3
www.wof-zw.de   1
jeanneowensauthor.wordpress.com   1
www.xn--gesnder-kochen-isb.de   1
www.xlase.de   1
www.versicherungsmagazin.de   2
stupidityhole.wordpress.com   1
www.workandplay-hh.de   1
stuffjeffreads.wordpress.com   1
reinfecta.wordpress.com   1
thmatters.wordpress.com   1
greensboroobserver.wordpress.com   2
twotwitchatale.wordpress.com   1
obonsfotoblogg.wordpress.com   1
headhunterconfesses.wordpress.com   1
wiwa-kampfmittelbergung.de   1
www.en.wollnywp.de   1
www.work-at-hair.com   1
vogtland-kartonagen.de   1
manojkandasamy.wordpress.com   1
nopeanuts.wordpress.com 

remotehrsolutions.wordpress.com   1
worklink-hr.com   1
visuelleconcepts.de   1
katsmiao.wordpress.com   1
susanpalmerconsulting.wordpress.com   1
workfromhomedata.com   1
www.villakunterbunt.wtal.de   1
thepersecutionofmildreddunlap.wordpress.com   1
monarchieundalltag.wordpress.com   1
www.vitalundfitmit100.de   1
wir-machen-aus-ideen-projekte.de   1
www.vg-aiterhofen.de   2
www.volksbanksulingen.de   1
newberrymediacorp.wordpress.com   1
greenlightfilm.wordpress.com   1
www.vb-mittelhessen.de   1
history2research.wordpress.com   1
www.wohnen-im-eigentum.de   2
workingclassproduction.com   1
www.worldcupplayoffsbracket.com   1
www.vierke.de   1
www.verdi8-hellweg.de   1
tinybobmac.wordpress.com   1
peoplesally.wordpress.com   1
poetscircle.wordpress.com   1
www.video-art.de   1
spornitz.vandervalk.de   1
www.willbrandt.de   1
globalyouthbook.wordpress.com   1
www.viva-akquise.de   1
lukelawrenceocr.wordpress.com   1
nafcenglish.wordpress.com   1
iftodaywehear.wordpress.com   1
triath

workscounselingcenter.com   1
veryuglyplates.de   1
kidlilly.wordpress.com   1
mitziwaltz.wordpress.com   1
www.xn--brggemann-design-kzb.de   2
www.veronika-zickendraht.de   1
psicosi2012.wordpress.com   1
sitecorecontextitem.wordpress.com   1
www.wordsworkers.com   1
www.wild-eppingen.de   1
www.workwada.com   1
www.wlp-systems.de   1
paulineobrien.wordpress.com   2
kalailmforlove.wordpress.com   1
lowmanhenry.wordpress.com   1
bkk-vdn.vion-online.de   1
talkiesnetwork.wordpress.com   1
rheingoldblog.wordpress.com   1
remissartist.wordpress.com   1
tomasclancy.wordpress.com   1
sonibindaas571181659.wordpress.com   1
readneckreview.wordpress.com   1


cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.workana.com' --fields url | wc -l

> 55

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.workopolis.com' --fields url | wc -l

> 0


cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.worknrby.com' --fields url | wc -l

> 0

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.workinsports.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://seeker.worksourcewa.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://kwoted.wordpress.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.xn--jobbrse-stellenangebote-blc.de' --fields url | wc -l

> 5

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://www.workitdaily.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://workinhostels.com' --fields url | wc -l

> 1

cdxt --cc --from 202101 --to 202102 --filter '=status:200' iter 'https://workshift-sol.com' --fields url | wc -l

> 1